In [214]:
!pip install requests
!pip install BeautifulSoup
!pip install pandas


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: d:\python\python 3.7.0\python.exe -m pip install --upgrade pip


  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [7 lines of output]
  Traceback (most recent call last):
    File "<string>", line 36, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Zeroxin\AppData\Local\Temp\pip-install-94vpbb9l\beautifulsoup_2a7bbc9130db4ac6b828275d74d591c1\setup.py", line 3
      "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                      ^
  SyntaxError: invalid syntax
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, n


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: d:\python\python 3.7.0\python.exe -m pip install --upgrade pip


In [215]:
#imports

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [216]:
url = 'http://quotes.toscrape.com/'

In [217]:
def GetPageContent(pageNumber: int):
    '''Возвращает содержимое html схемы на определенной странице'''
    return requests.get(f'{url}/page/{pageNumber}/')


def CreateHTMLParser(resp):
    return BeautifulSoup(resp.content, 'html.parser')


def GetAuthorPage(authorPageName: str):
    return requests.get(f'{url}{authorPageName}')


def GetQuotesPageByTagAndNumPage(tagName: str, pageNumber: int):
    return requests.get(f'{url}/tag/{tagName}/page/{pageNumber}/')

In [218]:
def GetTopTenTags(baseTree: BeautifulSoup):
    '''Собирает топ 10 тэгов на сайте http://quotes.toscrape.com/'''
    topTenTagsBlock = baseTree.find('div', {
        'class': 'col-md-4 tags-box'})  #получение целого блока со всеми тэгами в необработанном виде
    topTenTagsBlocks = topTenTagsBlock.find_all('a', {
        'class': 'tag'})  #получение списка самих тэгов в необработанном виде. они обозначенны как ссылки.

    topTenTags = []  #список названий тэгов

    for tag in topTenTagsBlocks:
        topTenTags.append(tag.text)

    return topTenTags


def GetQuotesCountByTag(tagName: str):
    pageNum = 1
    tagResponse = GetQuotesPageByTagAndNumPage(tagName, pageNum)

    quotesCount = 0

    while True: #обхожу страницы пока там не будет пусто (статус кода 404 там нет)
        treeTag = CreateHTMLParser(tagResponse)
        currentPageQuotesCount = len(GetQuotesSchemas(treeTag))

        if (currentPageQuotesCount == 0):
            break

        quotesCount += currentPageQuotesCount
        pageNum += 1
        tagResponse = GetQuotesPageByTagAndNumPage(tagName, pageNum)

    return quotesCount

In [219]:
def CreateQuoteDictionary(quoteTree: BeautifulSoup):
    quoteData = {}

    quoteData['Text'] = GetQuoteTextSchema(quoteTree).text.strip()
    quoteData['Author'] = GetQuoteAuthorName(quoteTree)
    quoteData['Tags'] = [x.text.strip() for x in
                         quoteTree.find('div', {'class': 'tags'}).find_all('a', {'class': 'tag'})]

    return quoteData

In [220]:
def GetQuotesSchemas(baseTree: BeautifulSoup):
    return baseTree.find_all('div', {'class': 'quote'})


def GetQuoteTextSchema(quoteTree: BeautifulSoup):
    return quoteTree.find('span', {'class': 'text'})


def GetQuoteAuthorSchema(quoteTree: BeautifulSoup):
    return quoteTree.find('span', {'class': ''})


def GetQuoteAuthorName(quoteTree: BeautifulSoup):
    return GetQuoteAuthorSchema(quoteTree).find('small', {'class': 'author'}).text.strip()


def GetQuoteAuthorHomePage(quoteTree: BeautifulSoup):
    return GetQuoteAuthorSchema(quoteTree).find('a', href=True)['href']

In [221]:
def GetAllQuotes(baseTree: BeautifulSoup):
    '''Возвращает все цитаты с информацией о тэгах, авторе и текстом цитаты'''
    quotes = GetQuotesSchemas(baseTree)

    quotesData = []

    for quote in quotes:
        quotesData.append(CreateQuoteDictionary(quote))

    return quotesData

In [222]:
def GetAuthorsHref(baseTree: BeautifulSoup):
    quotes = GetQuotesSchemas(baseTree)

    authorsHrefs = []

    for quote in quotes:
        authorsHrefs.append(GetQuoteAuthorHomePage(quote))

    return list(sorted(set(authorsHrefs)))

In [223]:
def GetAuthorsNames(baseTree: BeautifulSoup):
    quotes = GetQuotesSchemas(baseTree)

    authorsNames = []

    for quote in quotes:
        authorsNames.append(GetQuoteAuthorName(quote))

    return list(sorted(set(authorsNames)))

In [224]:
def GetAuthorHomePageData(baseAuthorTree: BeautifulSoup):
    authorDetails = baseAuthorTree.find('div', {'class': 'author-details'})

    authorData = {}

    authorData['Name'] = authorDetails.find('h3', {'class': 'author-title'}).contents[0].text.strip()
    authorData['Born Date'] = authorDetails.find('span', {'class': 'author-born-date'}).text.strip()
    authorData['Bord Location'] = authorDetails.find('span', {'class': 'author-born-location'}).text.strip()
    authorData['Description'] = authorDetails.find('div', {'class': 'author-description'}).text.strip()

    return authorData

In [225]:
def GetTopTenAuthors():
    quotes = []
    for page in range(1, 11):  #обход по циклу 10 страниц
        localResponse = GetPageContent(page)
        baseTree = CreateHTMLParser(localResponse)
        quotes += GetQuotesSchemas(baseTree)

    authorsList = []
    authors = []

    for quote in quotes:
        authors.append(GetQuoteAuthorName(quote))

    authorsSet = set(authors)

    for item in authorsSet:
        authData = {}
        authData['Author Name'] = item
        authData['Quotes Count'] = authors.count(item)
        authorsList.append(authData)

    return sorted(authorsList, key=lambda d: d['Quotes Count'], reverse=True)[:10]

Сбор цитат, находящихся на 1-10 страницах

In [226]:
tenPagesQuotes = []
authorsHomePages = []

for page in range(1, 11):  #обход по циклу 10 страниц
    response = GetPageContent(page)
    tree = CreateHTMLParser(response)
    tenPagesQuotes += GetAllQuotes(tree)
    authorsHomePages.append(GetQuoteAuthorHomePage(tree))

Вывод цитат, находящихся на 1-10 страницах

In [227]:
pd.DataFrame.from_dict(tenPagesQuotes)

,Text,Author,Tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,[inspirational]


Вся информация об авторах, найденных цитат на 1-10 страницах

In [228]:
authorsData = []

for authorHomePage in authorsHomePages:
    authorResponse = GetAuthorPage(authorHomePage)
    authorTree = CreateHTMLParser(authorResponse)
    authorsData.append(GetAuthorHomePageData(authorTree))

pd.DataFrame.from_dict(authorsData)

,Name,Born Date,Bord Location,Description
0,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,Marilyn Monroe,"June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
2,Pablo Neruda,"July 12, 1904","in Parral, Chile","Pablo Neruda was the pen name and, later, lega..."
3,Dr. Seuss,"March 02, 1904","in Springfield, MA, The United States",Theodor Seuss Geisel was born 2 March 1904 in ...
4,George R.R. Martin,"September 20, 1948","in Bayonne, New Jersey, The United States","George R. R. Martin was born September 20, 194..."
5,Jane Austen,"December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
6,Charles Bukowski,"August 16, 1920","in Andernach, Germany",Henry Charles Bukowski (born as Heinrich Karl ...
7,Alfred Tennyson,"August 06, 1809","in Somersby, Lincolnshire, The United Kingdom","Alfred Tennyson was born in Somersby, Lincolns..."
8,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
9,J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...


Вывод топ 10 авторов. (Пришлось придумывать велосипед, т. к. на сайте нет страницы с топами - такое только с тэгами есть)

In [229]:
pd.DataFrame.from_dict(GetTopTenAuthors())

,Author Name,Quotes Count
0,Albert Einstein,10
1,J.K. Rowling,9
2,Marilyn Monroe,7
3,Mark Twain,6
4,Dr. Seuss,6
5,C.S. Lewis,5
6,Jane Austen,5
7,Bob Marley,3
8,Mother Teresa,2
9,Ernest Hemingway,2


Вывод топ 10 тэгов без количества цитат

In [230]:
tags = GetTopTenTags(tree)
tags

['love',
 'inspirational',
 'life',
 'humor',
 'books',
 'reading',
 'friendship',
 'friends',
 'truth',
 'simile']

In [231]:
tagsData = []

for tag in tags:
    tagData = {}
    tagData['Tag Name'] = tag
    tagData['Quotes Count'] = GetQuotesCountByTag(tag)
    tagsData.append(tagData)

Вывод топ 10 тэгов с количеством цитат

In [232]:
pd.DataFrame.from_dict(tagsData)

,Tag Name,Quotes Count
0,love,14
1,inspirational,13
2,life,13
3,humor,12
4,books,11
5,reading,7
6,friendship,5
7,friends,4
8,truth,4
9,simile,3
